In [79]:
import pandas as pd
import numpy as np

# Get data files as pandas dataframes
particulate_matter_pd = pd.read_csv("datasets/particulate_matter.csv")
pitt_census_tracts_pd = pd.read_csv("datasets/Pittsburgh_Census_Tracts.csv", encoding='cp1252') # there were encoding errors with utf-8, I just guessed the encoding of the file

# Only want census tracts from year 2010
census_tracts_mask = pitt_census_tracts_pd['Year'] == 2010
pitt_census_tracts_pd = pitt_census_tracts_pd[census_tracts_mask]

# Converts the census tracts to floats (there may also be more than one tract per entry)
def split_and_float(tract):
    tract_split = tract.split(', ')
    final_arr = []
    for tract in tract_split:
        final_arr.append(float(tract))
    return final_arr
pitt_census_tracts_pd['FloatTracts'] = pitt_census_tracts_pd['Census Tracts'].apply(split_and_float)

# For some reason the census tracts in this file are multiplied by 100
particulate_matter_pd['RealTract'] = particulate_matter_pd['CensusTract'] / 100

# Identifies the Neighborhood of each census tract and attaches it to particulate matter dataframe (this is really slow but necessary)
def find_neighborhood(tract):
    for index, row in pitt_census_tracts_pd.iterrows():
        if tract in row['FloatTracts']:
            return row['Neighborhood']
    return None
particulate_matter_pd['Neighborhood'] = particulate_matter_pd['RealTract'].apply(find_neighborhood)
part_mask = particulate_matter_pd['Neighborhood'] != None
particulate_matter_pd = particulate_matter_pd[part_mask]

# We don't need the census tracts anymore now that the neighborhoods are matched up
particulate_matter_pd.drop(["CensusTract", "RealTract"], axis=1, inplace=True)

merge_guide = {'PM25': 'mean'}
particulate_matter_pd = particulate_matter_pd.groupby(particulate_matter_pd['Neighborhood']).aggregate(merge_guide)

print(len(particulate_matter_pd))
particulate_matter_pd.head(10)



77


,PM25
Neighborhood,
Allegheny Center,12.251408
Allentown,12.301099
Arlington,12.338299
Banksville,12.220888
Bedford Dwellings,12.243351
Beechview,12.284014
Beltzhoover,12.307153
Bloomfield,12.215839
Bluff,12.264655
